In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm/utils.py'>

In [4]:
dat1 = pd.read_csv('../lyrics_files/allSongs.csv').drop_duplicates()

In [5]:
dat2 = pd.read_excel('../lyrics_files/chancelyrics.xlsx')

In [6]:
dat3 = pd.read_csv('../lyrics_files/eminem.csv')

In [7]:
dat4 = pd.read_csv('../lyrics_files/fromWeb.csv')

In [8]:
dat5 = pd.read_csv('../lyrics_files/kanye_west_discography.csv')

In [22]:
dat6 = pd.read_csv('../lyrics_files/lyrics.csv')

In [23]:
dat6.head()

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."


In [24]:
dat = pd.concat([dat1.lyrics, dat2.Lyrics, dat3.text, dat4.Lyrics, dat5.lyric, dat6.lyrics])

In [25]:
dat.shape

(363605,)

In [26]:
glove = pd.read_table('../glove.6B.50d.txt', sep=' ', header=None, quoting=3, na_filter=False, index_col=0)

In [27]:
glove.shape

(400000, 50)

In [28]:
glove_words = set(glove.index.tolist())

In [29]:
dat += '(end)'

In [31]:
dat = pd.DataFrame(dat, columns=['text'])

In [32]:
dat.head()

,text
0,"I'm tryna put you in the worst mood, ah\nP1 cl..."
1,"Hey, I was doing just fine before I met you\nI..."
2,I used to believe\nWe were burnin' on the edge...
3,Baby I like your style\n\nGrips on your legs\n...
4,"Yeah, yeah\n\nI've been down so long it look l..."


In [46]:
cleaned = utils.data_curating(dat.iloc[:20000].dropna().copy())

In [47]:
cleaned.head()

0    i 'm tryna put you in the worst mood , ah \n p...
1    hey , i was doing just fine before i met you \...
2    i used to believe \n we were burnin ' on the e...
3    baby i like your style \n \n grips on your leg...
4    yeah , yeah \n \n i 've been down so long it l...
Name: cleaned_text, dtype: object

In [48]:
cleaned = cleaned.str.split(' ')

In [49]:
cleaned.head()

0    [i, 'm, tryna, put, you, in, the, worst, mood,...
1    [hey, ,, i, was, doing, just, fine, before, i,...
2    [i, used, to, believe, \n, we, were, burnin, '...
3    [baby, i, like, your, style, \n, \n, grips, on...
4    [yeah, ,, yeah, \n, \n, i, 've, been, down, so...
Name: cleaned_text, dtype: object

In [42]:
word2ind, ind2word = utils.load_index_word_map()

In [50]:
cleaned = cleaned.apply(lambda words: [word for word in words if word in word2ind.keys() or word == '\n'])

In [51]:
cleaned.shape

(14671,)

In [52]:
cleaned.head()

0    [i, 'm, tryna, put, you, in, the, worst, mood,...
1    [hey, ,, i, was, doing, just, fine, before, i,...
2    [i, used, to, believe, \n, we, were, burnin, '...
3    [baby, i, like, your, style, \n, \n, grips, on...
4    [yeah, ,, yeah, \n, \n, i, 've, been, down, so...
Name: cleaned_text, dtype: object

In [32]:
cleaned_sets = cleaned.apply(set)

In [33]:
vocab_size, unique_tokens = utils.get_unique_tokens(cleaned_sets)

In [34]:
vocab_size

53375

In [35]:
unique_tokens[:5]

['\n', '!', '"', "'", "'cause"]

In [36]:
utils.save_unique_tokens(unique_tokens)

In [37]:
word2ind, ind2word = utils.get_index_word_map(unique_tokens)

In [38]:
utils.save_index_word_map(word2ind, ind2word)

In [39]:
emb = utils.create_emb(ind2word, glove)

In [40]:
utils.save_embedding(emb)

In [53]:
tokenised = utils.tokenise_cleaned_data(cleaned, word2ind)

In [54]:
tokenised.head()

0    [22747, 8, 48963, 37155, 53102, 23217, 47371, ...
1    [21653, 19, 22747, 51500, 13568, 25135, 17121,...
2    [22747, 50260, 47949, 4072, 0, 51644, 51839, 6...
3    [2963, 22747, 27142, 53117, 45555, 0, 0, 20163...
4    [52967, 19, 52967, 0, 0, 22747, 16, 3944, 1385...
Name: cleaned_text, dtype: object

In [55]:
%%time
samples = utils.make_training_samples(tokenised, input_length=200)

CPU times: user 5min 7s, sys: 2.72 s, total: 5min 9s
Wall time: 5min 9s


In [56]:
samples.shape

(2090316, 201)

In [57]:
samples.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,22747,8,48963,37155,53102,23217,47371,52683,30536,19,...,48960,47949,47280,29193,19,838,0,15862,11661,36
1,8,48963,37155,53102,23217,47371,52683,30536,19,838,...,47949,47280,29193,19,838,0,15862,11661,36,31874
2,48963,37155,53102,23217,47371,52683,30536,19,838,0,...,47280,29193,19,838,0,15862,11661,36,31874,48960
3,37155,53102,23217,47371,52683,30536,19,838,0,33532,...,29193,19,838,0,15862,11661,36,31874,48960,47949
4,53102,23217,47371,52683,30536,19,838,0,33532,23,...,19,838,0,15862,11661,36,31874,48960,47949,15192


In [58]:
utils.save_training_samples(samples, 'train2.csv')